In [ ]:
# - Cleanup
# - remove "magic literals"
# - remove lambdas
# - add types
# - Add docstring

import typing
import pandas as pd

def size_name(df_, name=''):
    print(f'{name} {df_.shape}')
    return df_

def tweak_ames(ames:pd.DataFrame) -> pd.DataFrame:
    """ Clean up raw Ames data ...
    """

    def rating_to_num(a_df:pd.DataFrame,col:str) -> pd.Series:
        return a_df[col].replace({'Ex':5, 'Gd':4, 'TA':3,'Fa':2,'Po':1})

    def create_rating_fn(col:str) -> typing.Callable:
        def wrapper(a_df):
            return rating_to_num(a_df,col)
        return wrapper

    def clean_col_name(col:str) -> str:
        return col.lower().replace(' ','_')

    def extract_story_num(a_df: pd.DataFrame) -> pd.Series:
        return a_df.house_style.str.extract(r'(\d\.?\d?)').astype(float).fillna(0)
    
    def has_basement(a_df: pd.DataFrame) ->pd.Series:
        return ~a_df.bsmt_qual.isna()

    def has_garage(a_df: pd.DataFrame) ->pd.Series:
        return ~a_df.garage_qual.isna()

    def fix_garage_year(a_df: pd.DataFrame) ->pd.Series:
        return a_df.garage_yr_blt.fillna(a_df.year_built).astype('unit8')

    def create_category_fillna_fn(col:str) =>typing.Callable:
        def wrapper(a_df: pd.DataFrame) ->pd.Series:
            return a_df[col].fillna('Other').astype('category')
        return wrapper
    
    def create_float_fillna_fn(col:str) -> typing Callable:
        def wrapper(a_df: pd.DataFrame) -> pd.Series:
            return a_df[col].fillna(0).astype(float)
        return wrapper
    
    rating_cols=['exter_qual','exter_cond','bsmt_qual',
                  'bsmt_cont', 'kitchen_qual', 'garage_qual',
                  'garage_cond', 'heating_qc', 'fireplace_qu'
                ]

    rating_mapping = {k:create_rating_fn(0)
                        for k in rating_cols}

    category_cols = ['ms zoning'   , 'street'        , 'alley'           , 'lot_shape'    ,   'land_contour',
                    'utilities'    , 'lot_config'    , 'land_slope'      , 'neighborhood' ,   'condition_1',
                    'condition_2'  ,  'bldg_type'    , 'house_style'     , 'roof_style'   ,   'roof_mtl',
                    'exterior_lst' , 'exterior_2nd'  , 'mas_vnr_type'    , 'foundation'   ,
                    'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_type_2'  , 'heating'      , 
                    'electrical'   , 'functional'    , 'garage_type'     , 'garage_finish',
                    'paved_drive'  , 'fence'         ,  'misc_feature'   , 'sale_type'    ,    'sale_condition']

    category_mapping = {k:create_category_fillna_fn(k) for k in category_cols}
    